PROJECT CODE

TASKS:

1_Expansão com hidrolases - MEME and BLAST
2_Features - NS Machine learning - Clustering and Kmers
3_Functional Analysis - MOFA 
4_Estructural - Ensemble 

O que posso aplicar para a generalidade dos casos?


In [37]:
# Blast

from Bio.Blast.Applications import NcbiblastpCommandline

# Arquivos de entrada
initial_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta"
new_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/new_sequences.fasta"
output_file = "/Users/Sergiomendes/Desktop/Project/data/blast_results.tsv"

# Remover o arquivo de saída, se existir
if os.path.exists(output_file):
    os.remove(output_file)

# Executar o BLAST usando o package do Anaconda e gravar os resultados no arquivo blast_results.tsv
blastp_cline = NcbiblastpCommandline(query=new_sequences_file, subject=initial_sequences_file, outfmt=6, out=output_file)
stdout, stderr = blastp_cline()


In [28]:
#MEME

# Arquivos de entrada
initial_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta"
new_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/new_sequences.fasta"
output_file = "/Users/Sergiomendes/Desktop/Project/data/meme_results.txt"

import subprocess

# Comando para executar o programa meme e redirecionar a saída para o arquivo
command = ["/opt/local/bin/meme", initial_sequences_file, new_sequences_file, "-nmotifs", "3", "-oc", output_file]

# Executar o comando
subprocess.run(command)

print("Análise MEME concluída. Os resultados foram salvos no arquivo meme_results.txt.")



Análise MEME concluída. Os resultados foram salvos no arquivo meme_results.txt.


In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

initial_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta"
new_sequences_file = "/Users/Sergiomendes/Desktop/Project/data/new_sequences.fasta"

def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    sequences.append(sequence)
                sequence = ''
            else:
                sequence += line
        if sequence:
            sequences.append(sequence)
    return sequences

# Retornar os encoders de cada sequencia
def encode_sequence(sequence):
    encoding = []
    for amino_acid in sequence:
        encoding.append(ord(amino_acid))
    return encoding

file_paths = [initial_sequences_file, new_sequences_file]

encoders_file1 = []
encoders_file2 = []

for file_path in file_paths:
    sequences = read_fasta(file_path)
    encoders = []
    for sequence in sequences:
        encoding = encode_sequence(sequence)
        encoders.append(encoding)

    if file_path == initial_sequences_file:
        encoders_file1 = encoders
    elif file_path == new_sequences_file:
        encoders_file2 = encoders

all_encoders = encoders_file1 + encoders_file2

# Preencher as sequências com zeros para terem o mesmo comprimento
padded_sequences = pad_sequences(all_encoders, padding='post')

# Converter a lista de encoders em uma matriz numpy de ponto flutuante
data = np.array(padded_sequences, dtype=np.float)

pca = PCA(n_components=2)
principal_components = pca.fit_transform(data)

kmeans = KMeans(n_clusters=2, random_state=0)
clusters = kmeans.fit_predict(principal_components)

plt.scatter(principal_components[:, 0], principal_components[:, 1], c=clusters)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Clustering usando K-means')
plt.savefig('clustering_result.png')
plt.close()


<ipython-input-1-70bbc411f0b0>:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.array(padded_sequences, dtype=np.float)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:

"""
# PCA & unsupervised K-means (A adaptar)

# resfinder matriz 01
tudodegenes = []
for row in range(len(comb_file['resfinder'])):
    rf_ls = comb_file['resfinder'][row]
    tudodegenes.append(rf_ls)
flat_tudodegenes = [x for xs in tudodegenes for x in xs]
final_tg = list(dict.fromkeys(flat_tudodegenes))

# add all zeros
for i in final_tg:
    comb_file.insert(loc=9, column=i, value=0)
# if hit = 1
final_resfinder = comb_file.set_index('id')
id_rf = dict(zip(comb_file.id, final_resfinder['resfinder']))
for key, value in id_rf.items():
    for i in value:
        for z in final_resfinder.columns[8:]:
            if i == z:
                final_resfinder.loc[key, z] += 1
            else:
                continue

# standardização dos dados
scaler = StandardScaler()
final_resfinder_scaled = scaler.fit_transform(final_resfinder[final_tg])

# Pca
rf_PCA = PCA()
rf_PCA.fit(final_resfinder[final_tg])
rf_pca_reduced = rf_PCA.transform(final_resfinder[final_tg])


for x in final_resfinder["type"].unique():
    sp = comb_file.index[final_resfinder["type"] == x]
    if x == "Agricultural" or x == "Freshwater":
        plt.plot(rf_pca_reduced[sp, 0], rf_pca_reduced[sp, 1], "X", label=x, markersize=8)
    else:
        plt.plot(rf_pca_reduced[sp, 0], rf_pca_reduced[sp, 1], "o", label=x, markersize=8)

 add dashed lines for zero lines
plt.axhline(y=0, color='gray', linestyle='--', linewidth=0.5)
plt.axvline(x=0, color='gray', linestyle='--', linewidth=0.5)

plt.title("ResFinder: ARGs PCA")
plt.xlabel("P.C. 1: " + str(rf_PCA.explained_variance_ratio_[0])[0:4])
plt.ylabel("P.C. 2: " + str(rf_PCA.explained_variance_ratio_[1])[0:4])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# set x-axis limits
plt.xlim([-6, 16])
plt.xticks(range(-5, 16, 5))

plt.show()

# k means clustering resfinder
from sklearn.cluster import KMeans

data = final_resfinder[final_tg]
pca = PCA(2)
df_1 = pca.fit_transform(data)

kmeans = KMeans(n_clusters=4, max_iter=1000)
label = kmeans.fit_predict(df_1)
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)
for i in u_labels:
    plt.scatter(df_1[label == i, 0], df_1[label == i, 1], label=i)
plt.scatter(centroids[:, 0], centroids[:, 1], s=80, color='k')
plt.title("Resfinder kmeans")
plt.legend()
plt.show()

# optimal kmeans resfinder
Sum_of_squared_distances = []
K = range(1, 15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_1)
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k: Resfinder')
plt.show()

final_resfinder.to_excel("C:/Users/diogo/Desktop/rf_01.xlsx")

"""

In [ ]:
# FUNCTIONAL ANALYSIS - MOFA

import numpy as np
from mofapy2.mofapy2 import MOFA
from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder

# Carregar as sequências de proteínas do arquivo initial_sequences.fasta
initial_sequences = []
with open(inicial_sequences_file, "r") as initial_seq_file:
    for record in SeqIO.parse(initial_seq_file, "fasta"):
        initial_sequences.append(str(record.seq))

# Carregar as sequências de proteínas do arquivo new_sequences.fasta
new_sequences = []
with open(new_sequences_file, "r") as new_seq_file:
    for record in SeqIO.parse(new_seq_file, "fasta"):
        new_sequences.append(str(record.seq))

# Codificar as sequências de proteínas usando One-Hot Encoding
encoder = OneHotEncoder(dtype=np.int8)
protein_sequences = initial_sequences + new_sequences
encoded_sequences = encoder.fit_transform(protein_sequences).toarray()

# Preparar as características funcionais relevantes 
# Certificar que as características funcionais estão no formato adequado (variáveis numéricas ou categóricas)

# Criar a matriz de dados para a análise MOFA
X = np.array(encoded_sequences)

# Configurar e executar o MOFA
model = MOFA()
model.set_data_matrix(X)
model.fit(n_factors=3)  # Definir o número de fatores latentes desejado

# Analisar os resultados do MOFA
factors = model.get_factors()
loadings = model.get_loadings()

# Exemplo de impressão dos resultados
for factor_idx, factor in enumerate(factors):
    print(f"Factor {factor_idx + 1}:")
    print(factor)
    print("")

for loading_idx, loading in enumerate(loadings):
    print(f"Loading {loading_idx + 1}:")
    print(loading)
    print("")



In [ ]:
# ESTRUCTURAL ANALYSIS - ENSEMBLE

In [ ]:
# EXPANSÃO DO DATASET

import os
from Bio import SeqIO
import requests

# Define os termos de busca e filtra por proteínas da classe hidrolase
search_term = 'hidrolase NOT "PET degradation"'
search_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=protein&term={search_term}&retmax=100000'
search_result = requests.get(search_url).text
id_list = search_result.split('<Id>')[1:]
id_list = [id.split('</Id>')[0] for id in id_list]

# Baixa as sequências das proteínas encontradas
fetch_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&id={",".join(id_list)}&rettype=fasta&retmode=text'
sequences = SeqIO.parse(requests.get(fetch_url).text, 'fasta')

# Grava as sequências em um arquivo FASTA na pasta desejada
output_folder = '/Users/Sergiomendes/Desktop/Project/Code/data'
output_file = os.path.join(output_folder, 'proteins_not_degrading_pet.fasta')
with open(output_file, 'w') as f:
    SeqIO.write(sequences, f, 'fasta')



In [ ]:
# JUNTAR OS FICHEIROS FASTA CRIANDO UM DATASET EXPANDIDO DE HIDROLASES

# Define os caminhos para os arquivos FASTA que serão combinados
file1 = '/Users/Sergiomendes/Desktop/Project/data/initial_sequences.fasta'
file2 = '/Users/Sergiomendes/Desktop/Project/data/proteins_not_degrading_pet.fasta'

# Lê as sequências de ambos os arquivos
seqs1 = SeqIO.parse(file1, 'fasta')
seqs2 = SeqIO.parse(file2, 'fasta')

# Combina as sequências em uma lista
combined_seqs = list(seqs1) + list(seqs2)

# Define o caminho para o arquivo de saída
output_folder = '/Users/Sergiomendes/Desktop/Project/data'
output_file = os.path.join(output_folder, 'expanded_dataset.fasta')

# Grava as sequências combinadas em um único arquivo FASTA
with open(output_file, 'w') as output_handle:
    SeqIO.write(combined_seqs, output_handle, 'fasta')